# TP 2 : Cryptographie

Dans ce TP on étudiera l'algorithme RSA ainsi que des divers algorithmes de primalité.

Voici une référene qui pourra être utile pour des astuces avec Python : https://www.kaggle.com/learn/python


# 1) RSA

Rappelons que système RSA (d'après Rivest, Shamir et Adleman, ses créateurs) est bassé sur le principe de fonction à sens unique. Dans ce cas-ci, la fonction est :

\begin{equation} f : \mathbb{N} \times \mathbb{N} \to \mathbb{N} \;\;\; : \;\;\; (a,b) \mapsto ab, \end{equation}

dont l'inverse implique factoriser un nombre $n$ comme produit des deux nombres $a,b > 1$, ce qui est un problème difficile, comme on le verra dans le deuxième exercice.

L'objectif de cet exercice est d'implementer le système RSA. Rappelons les notations :

*   $p$ et $q$ sont deux nombres premiers.
*   $n =pq$ de sorte que $\phi(n) = (p-1) (n-1)$.
*   $1 \leq e < \phi(n)$ est un nombre choisit au hasard et premier à $\phi(n)$. 
*   $1 \leq d < \phi(n)$ est l'inverse de $e$ modulo $\phi(n)$.

Alors la clé publique est donnée par $(n, e)$ et la clé sécrète par $(p,q,d)$.

a) Rappeler l'algorithme du système RSA.

b) Écrire une fonction qui retourne une clé publique à partir de $(p, q)$

In [14]:
# La fonction public_key prend comme argument deux nombres premiers p et q et retourne une clé publique public_key = (n, e)
# et une clé secrète secret_key = (p, q, d)
import random
import math
def rsa_keys(p, q) :
    n=p*q
    phi=(p-1)*(q-1)
    while(True):
        e=random.randint(1,phi)
        if math.gcd(e,phi) == 1 :
            d=pow(e, -1, phi)
            break
    return (n,e,d)
print(rsa_keys(7,11))
    

(77, 43, 7)


c) Écrire une fonction qui codifie un message $m$ (un entier) étant donné une clé publique $(n, e)$.

In [9]:
# La fonction crypt_rsa prend comme argument un entier m, la clé publique (n, e)
# et retourne le message codée
def crypt_rsa(m, n, e) :
    return (m**e) % n

print(crypt_rsa(5,7,2))

4


d) Écrire une fonction qui décode un message $m$ (un entier) crypté en utilisant la clé publique $(n, e)$ étant donné la clé secrète $(\varphi(n), d)$.

In [116]:
# La fonction decrypt_rsa prend comme argument m et retourne le message décrypté
def decrypt_rsa(m, n, d) :
    return (m**d) % n
print(decrypt_rsa(9,6,7))

3


e) Écrire une fonction pour coder un message $s$ (un texte) en utilisant une clé publique publique $(n, e)$.

In [29]:
# rsa_code prend comme argument un string s et retourne un message codé avec la clé publique (n, e)
def rsa_code(s, n, e) :
    code=[]
    message = []
    for i in range(len(s)):
        message.append(ord(s[i])-97)
        code.append(chr(crypt_rsa(message[i], n, e)))
    return code

print(rsa_code('aaaaaa',2,2))

['\x00', '\x00', '\x00', '\x00', '\x00', '\x00']


f) Écrire une fonction pour decoder un message $s$ (un texte) crypté en utilisant une clé publique $(n, e)$ et clé secrète $(\varphi(n), d)$.

In [117]:
# rsa_decode prend comme argument un message codé s (un string) avec la clé publique (n, e) et retourne le
# message décodé avec la clé secrète (phi, d) 
def rsa_decode(s, n, d) :
    decode=[]
    message_co = []
    for i in range(len(s)):
        message_co.append(ord(s[i])-97)
        decode.append(chr(decrypt_rsa(message_co[i], n, d)))
    return decode
#decode = ''
#for i in range(len(s)):
    #decode += chr()

print(rsa_decode('aaaaaa',2,2))

['\x00', '\x00', '\x00', '\x00', '\x00', '\x00']


g) Donner un exemple en utilisant p = 17, q = 23.

# 2) Tests de primalité

Les nombres premiers sont à la base du système RSA. On a besoin de pouvoir construire des nombres premiers grands et on s'intéresse donc à la question de quand est-ce un nombre $n$ premier ou composé.

## 2.1) Solovay-Strassen

Implementer l'algorithme de Solovay-Strassen et l'utiliser pour donne la liste des 100 premiers nombres premiers.

In [56]:
def Jacobi(M,N):
    """
    Calcule le symbole s = (M/N) par la mÃ©thode vue en TD.
    """
    assert(N%2 == 1)
    m, n, s = M, N, 1
    if (m < 0):
        m = -m
        s = 1 if (((n-1)//2)%2==0) else -1
    while(m >= 2):
        if (m % 2 == 0):
            m //= 2
            exposant = (n**2 -1) // 8
            s *= 1 if exposant%2 == 0 else -1
        else:
            exposant = (n-1)*(m-1)//4
            s *= 1 if exposant%2 == 0 else -1
            n, m = m, n%m
    if m == 0:
        return 0
    return s
Jacobi(6,7)

-1

In [111]:
# La fonction solovay_strassen prend comme argument deux entiers n et k. Elle doit choisir k nombres differents aléatoirement et tester la condition
# de Sollovay-Strassen. Si n n'est pas premier, le programme doit écrire "n est compossé", si n passe k tests, le programme doit répondre
# "n est un nombre premier avec probabilité p", où p est la probabilité de que n soit premier.


def solovay_strassen(n, k) :
    
    assert (n%2)==1
    while(k!=0):
        k=k-1
        a=random.randint(2,n-1)
        x=Jacobi(a,n)
        if (x==0) or ((x%n)!=pow(a,(n-1)//2,n)):
            print(n ,"est composé")
            break 
        if k==0:
            print(n, "est probablement premier avec probabilité")
solovay_strassen(2**(23)-1,100)
        


8388607 est composé


# 2.2) Rabin-Miller

Rappelons l'idée derrière l'algorithme de Rabin-Miller. Soit $n \in \mathbf{N}$ un entier impaire. On peut alors écrire $n$ sous la forme $1 + 2^k q$, où $k \geq 1$ et $q$ est un entier impaire. Si $n$ est un nombre premier, alors, pour tout entier $a$ tel que $(a, p) \equiv 1 \text{(mod $p$)}$, le petit théorème de Fermat dit que

\begin{equation} a^{2^k q} \equiv 1 \text{(mod $n$)}. \end{equation}

Soit $0 \leq i \leq k$ le plus petit entier tel que $a^{2^i q} \equiv 1 \text{(mod $n$)}$. On a alors deux possibilités :

*   Si $i =0$ alors $a^q \equiv 1 \text{(mod $n$)}$.
*   Si $i \geq 1$ alors $a^{2^{i-1} q} \equiv -1 \text{(mod $n$)}$.

Observez que la deuxième condition est vraie car il n'y a que deux racines carrées de 1 quand $n$ est premier (pourquoi ?).

**Definition** On dit que $n = 1 + 2^k q$ est *fortement pseudo-premier* de base $a$ pour $1 \leq a < n$ si l'une des connditions suivantes est satisfaite:


*   $a^q \equiv 1 \text{(mod $n$)}$
*   Il existe $0 \leq i \leq k-1$ tel que $a^{2^{i} q} \equiv -1 \text{(mod $n$)}$.

**Théorème** (Rabin-Miller) Soit

$B_n := \{ a \in (\mathbb{Z} / n \mathbb{Z})^\times : n \text{ est fortement pseudo-premier de base } a\}.$

Alors si $n$ n'est pas premier on a $\frac{|B_n|}{\phi(n)} \leq \frac{1}{4}$.

Ceci permet de procéder de la même manière qu'on a fait avec l'algorithme de Solovay-Strassen : on choisit $k$ entiers au hasard et on vérifie si $n$ est fortement pseudo-premier de base $k$. Si $n$ ne l'est pas, alors $n$ n'est pas un nombre premier. Si $n$ passe les $k$ tests, alors $n$ est premier avec une probabilité $1 - 1 / 4^k$.

a) Implementer l'algorithme de Rabin-Miller.

In [119]:
# La fonction rabin_miller prend comme argument deux entiers n et k. Elle doit choisir k nombres differents aléatoirement et tester la condition
# de Miller_Rabin. Si n n'est pas premier, le programme doit écrire "n est compossé", si n passe k tests, le programme doit répondre
# "n est un nombre premier avec probabilité p", où p est la probabilité de que n soit premier.

def rabin_miller(n, k) :
    if n == 2 or n ==3 :
       return True
    if n % 2 == 0:
       return False
    r, s = 0, n - 1
    while s % 2 == 0:
       r += 1
       s //= 2                     # la partie non paire de n-1 / permet la factorisation en nombre premier
    for _ in range(k):
      a = random.randint(2, n - 1)     
      x = pow(a, s, n)            
      if x == 1 or x == n - 1:
         continue                
      for _ in range(r - 1):      
         x = pow(x, 2, n)        
         if x == n - 1:
            break               
         else:
           return False            
    return True  
print(rabin_miller(7,11))

True


Si l'hypothèse de Riemann généralisée est vraie, alors il est possible de démontrer que, si $n$ est composé, alors il existe $a \leq 2 (\log n)^2$ qui ne satisfait pas le test de Rabin-Miller.

b) Implementer un algorithme détérministe sous l'hypothèse de Riemann généralisée.

In [ ]:
# La fonction rabin_miller_grh prend comme argument deux entiers n et k. Elle doit retourner "n est compossé" ou
# "Si l'hypothèse de Riemann généralisée est vrai, alors est un nombre premier !".

def rabin_miller_grh(n, k) :

c) Que dit l'hypothèse de Riemann généralisée ?

d) Est-ce 6277101735386680763835789423207666416083908700390324961279 un nombre premier ?

e) Un nombre premier de Fermat est un nombre premier de la forme $2^k + 1$. Trouver les premiers nombres premiers de Fermat.

f) Un nombre premier de Mersenne est un nombre premier de la forme $M_n = 2^n - 1$. Montrer que si $M_n$ est un nombre premier alors $n$ l'est aussi. Trouver les premiers nombres premiers de Mersenne.

## 2.3) L'algorithme p-1 de Pollard

Rappelons l'idée de l'algorithme. Soit $n \in \mathbb{N}$ et supposons que $p$ est un nombre premier divisanr $n$. Écrivons
\begin{equation} p-1 = q_1^{a_1} . \ldots . q_k^{a_k}\end{equation}
avec $q_i$ des nombres premiers et $a_1 > 0$. Soit $b \in \mathbb{N}$ tel que $q_i^{a_i} \leq b$ pour tout $1 \leq i \leq k$. On dit alors que $n$ est $b$-superfriable. Soit
\begin{equation} M(b) := ppcm(2, \ldots, b) = \prod_{q < b} q^{\lfloor \log_q b \rfloor}. \end{equation}
Par hypothèse, on a $p - 1 | M(b)$ et donc $2^{M(b)} \equiv 1 \text{ (mod $p$)}$ (pourquoi ?) ce qui implique $p | 2^M(b) - 1$. On conclut que $ppcm(2^{M(b)}-1, n)$ est un facteur non trivial de $n$.

Ceci implique que, sous l'hypothèse que $n$ est $b$-superfriable, on trouve un facteur non-trivial de $n$. Par contre, il est possible que le résultat du calcul précédent soit $n$ tout entier : par exemple, si $n$ est sans carrés et si tout nombre premier $p$ divisant $n$ est $b$-superfriable.

a) Donner des exemples de $n$ et $b$ tels que $ppcm(2^{M(b)}-1, n) = n$.

b) Implementer l'algorithme $p-1$ de Pollard.

In [3]:
# la fonction pollard prend comme argument un enter n et un entier b et détérmine
# si il est possible de calculer un facteur de n avec l'aogirithme de Pollard.

import numpy as np
def pollard(n, b):
    x=np.random.randint(1,n-1)
    if np.gcd(x,n)==1:
        y=x**(b-1)
        d=np.gcd(y,n)
        if (d==1 or d==n):
             print("echec")
        else:
            return d
pollard(9,3)    

echec


## 2.4) L'algorithme $\rho$ de Pollard


L'idée principale derrière l'algorithme est que si $p$ est le plus petit nombre premier divisant $n$, et si l'on prend $k$ nombres au hasard entre $0$ et $p-1$, alors deux d'entre eux seront égaux avec probabilité $\geq 1/2$ dès que $k \geq \sqrt{p}$. Comme $p$ n'est pas connu, on considère une suite $a_1, a_2, \ldots$ d'entiers mod $n$ que l'on produit par la formule de récurrence

$\begin{equation} a_1 = \sqrt{n}, \;\;\; a_{i+1} = a_i^2 + c \end{equation}$

pour un certain entier $c$. On considère la suite $(a_i \text{ mod } p)$ (qui n'est pas à priori connue). Cette suite est considérée comme une suite aléatoire et forcement périodique. 

a) Montrer qu'il existe des nombres $\mu$ et $\lambda$ tels que, pour tout $i \geq \mu$, $a_{i} \equiv a_{i + \lambda} \text{ (mod p)}$.

b) Montrer que $i = k \lambda \geq \mu$ si et seulement si $a_i \equiv a_{2i} \text{ (mod p)}$.

c) Implementer l'algorithme $\rho$ de Pollard.

In [ ]:
# La foncction pollard prend comme argument un entier n et retourne un facteur non-trivial de n ou bien
# "pas de facteur trouvé".

def pollard(n, c) :